In [19]:
import warnings

warnings.simplefilter(action='ignore', category=Warning)

import pandas as pd
import numpy as np

train_df = pd.read_csv('Datasets/train.csv')
test_df = pd.read_csv('Datasets/test.csv')

In [20]:
name_df = train_df['Name']
titles = name_df.apply(lambda x: x.split(',')[1].split('.')[0].strip())
title_list = np.unique(titles)

train_df['titles'] = titles


title_grp = train_df.groupby('titles')

title_age_dict = {}

for title in title_list:
    title_age_dict[title] = title_grp.get_group(title)['Age'].mean()

title_age_dict = {k:round(title_age_dict[k], 1) for k in title_age_dict}
#title_age_dict

In [21]:
'''
now replacing all NaN values in age column
'''
""" for title in title_list:
    ages = title_grp.get_group(title)['Age']
    if ages.hasnans == True:
        ages = ages.fillna(title_age_dict[title])
        train_df['Age'].loc[ages.index.values] = ages.values """

" for title in title_list:\n    ages = title_grp.get_group(title)['Age']\n    if ages.hasnans == True:\n        ages = ages.fillna(title_age_dict[title])\n        train_df['Age'].loc[ages.index.values] = ages.values "

In [22]:
def df_cleaner(df):
    df['Sex'] = df['Sex'].replace({'male': 1, 'female': 2})

    name_df = df['Name']
    titles = name_df.apply(lambda x: x.split(',')[1].split('.')[0].strip())
    title_list = np.unique(titles)

    df['titles'] = titles

    title_grp = df.groupby('titles')
    for title in title_list:
        ages = title_grp.get_group(title)['Age']
        if ages.hasnans == True:
            ages = ages.fillna(title_age_dict[title])
            df['Age'].loc[ages.index.values] = ages.values

    return df

In [23]:
train_df = df_cleaner(train_df)
test_df = df_cleaner(test_df)

In [24]:
x_train_df = train_df.iloc[:, [2, 4, 5]].values

y_train_df = train_df.iloc[:, [1]].values

x_test_df = test_df.iloc[:, [1, 3, 4]].values

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train_df, y_train_df, test_size = 0.25, random_state = 0)


In [25]:
from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train) 

from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)

from sklearn.metrics import accuracy_score

print ("Accuracy : ", accuracy_score(y_test, y_pred))

Accuracy :  0.3721973094170404


In [26]:
x_test = sc_x.transform(x_test_df)

y_pred = classifier.predict(x_test)

In [27]:

submission = pd.DataFrame(test_df['PassengerId'])
submission['Survived'] = y_pred

submission.to_csv('submission_naive_bayes.csv', index=False)